# --- Day 5: If You Give A Seed A Fertilizer ---

https://adventofcode.com/2023/day/5

In [1]:
class IdentityDict(dict):
    __missing__ = lambda self, key : key

test = IdentityDict()
test[1]   

1

In [2]:
import re

## Parse the Input Data

In [22]:
def parse_input(filename):
    """Parse input data for puzzle.

    Parameters
    ----------
    filename : str
        The name of the *.txt file in the inputs/ directory.

    Returns
    -------

    """
    maps = {}
    with open(f'../inputs/{filename}.txt') as _file:
        f = _file.readlines()
        print(f)
        for i in range(len(f)):
            if i == 0:
                seeds = [int(s) for s in re.findall("\d+", f[i])]

            if f[i].startswith("seed-to-soil map:"):
                maps["seed2soil"] = []
                i += 1


            if f[i].startswith("soil-to-fertilizer map:"):
                maps["soil2fert"] = []
                continue

            if f[i].startswith("fertilizer-to-water map:"):
                maps["fert2water"] = []
                continue

            if f[i].startswith("water-to-light map:"):
                maps["water2light"] = []
                continue

            if f[i].startswith("light-to-temperature map:"):
                maps["light2temp"] = []
                continue

            if f[i].startswith("temperature-to-humidity map:"):
                maps["temp2hum"] = []
                continue

            if f[i].startswith("humidity-to-location map:"):
                maps["hum2loc"] = []
                continue

    return seeds, maps

In [23]:
parse_input("test_almanac")

['seeds: 79 14 55 13\n', '\n', 'seed-to-soil map:\n', '50 98 2\n', '52 50 48\n', '\n', 'soil-to-fertilizer map:\n', '0 15 37\n', '37 52 2\n', '39 0 15\n', '\n', 'fertilizer-to-water map:\n', '49 53 8\n', '0 11 42\n', '42 0 7\n', '57 7 4\n', '\n', 'water-to-light map:\n', '88 18 7\n', '18 25 70\n', '\n', 'light-to-temperature map:\n', '45 77 23\n', '81 45 19\n', '68 64 13\n', '\n', 'temperature-to-humidity map:\n', '0 69 1\n', '1 0 69\n', '\n', 'humidity-to-location map:\n', '60 56 37\n', '56 93 4']


([79, 14, 55, 13],
 {'seed2soil': [],
  'soil2fert': [],
  'fert2water': [],
  'water2light': [],
  'light2temp': [],
  'temp2hum': [],
  'hum2loc': []})

## Part 1
---

In [ ]:
def solve1(seeds, maps):
    min_loc = float("inf")
    map_keys = ["seed2soil", "soil2fert", "fert2water", "water2light", "light2temp", "temp2hum", "hum2loc"]

    for seed in seeds:
        for key in map_keys:
            for line in maps[key]:
                pass

            
        


        
        
        min_loc = min(min_loc, loc)

    return min_loc

### Run on Test Data

### Run on Input Data

## Part 2
---

### Run on Test Data

### Run on Input Data